**Imports**

PyTorch Geometric

In [1]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 4.2 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 1.4 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.5 MB 3.8 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 750 kB 1.3 MB/s 
     |████████████████████████████████| 407 kB 4.3 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=b30cf1ecb682e22576088d2f9cdc4fa6a5d20eca2ab9a764e691c6fada10696e
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [14]:
import pandas as pd
import numpy as np

from torch_geometric.data import InMemoryDataset, download_url, Data
from datetime import datetime

**Mount GDrive**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ls

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
action_types.csv              participants_activity_anomalies.csv
activities.csv                patients.csv
activity_types.csv            scores.csv
anomalies.csv                 scores_predictions.csv
anomaly_types.csv             sensor_locations.csv
diagnosis_predictions.csv     sensors.csv
diagnosis_types.csv           tasks.csv
events.csv                    task_types.csv
overt_error_action_types.csv  trajectory_anomalies.csv


In [109]:
def task_activity_sensors(tasks, events, sensors, activities, patient):

  # Seleziona i task-activity eseguiti dal paziente 'patient'
  task_activity = tasks.query('patient == ' + patient)

  # Seleziona il tempo di inizio e fine delle attività svolte dal paziente 'patient'
  activities_patient = activities.query('patient == ' + patient)
  start_time = activities_patient.iloc[0]['start']
  end_time = activities_patient.iloc[len(activities_patient) - 1]['end']

  # Query nel dataframe events per selezionare solo i sensori utilizzati nelle attività
  sensors_events = events.query('patient == ' + patient + ' and time >= "' + start_time + '" and time <= "' + end_time + '"')

  # Matrice risultato che contiene i dati dei nodi
  res_node = pd.DataFrame(columns=['features'])
  res_edge = pd.DataFrame(columns=['times']) 

  # Dimensione dell'array delle features
  dim_features = len(sensors)

  # Dizionario che contiene la posizione del sensore nell'array delle features
  dict_sens = {}

  for i in range(0, dim_features):
    dict_sens[sensors.iloc[i]['sensor_id']] = i

  # Indice del task-activity
  i = 0

  # Numero edge del grafo
  num_edge = 0

  # Nome del task-activity precedente
  pair_str_prev = ""

  # Flag per salvataggio features di nodo e arco
  ins_upd = False
  last_node = False

  for i in range(0, len(task_activity)):

    # Controlla se il task-activity attuale è l'ultimo
    if ((i + 1) >= len(task_activity)):
      last_node = True

    # Nome del task-activity attuale
    pair_str = str(task_activity.iloc[i]['task']) + '-' + str(task_activity.iloc[i]['activity'])

    # Controlla se il task-activity attuale e quello precedente sono diversi
    if pair_str != pair_str_prev:

      # Viene salvato lo start time
      s_time = task_activity.iloc[i]['time']

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str

        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str
    
    else:
      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Controlla se il task-activity attuale e quello successivo sono diversi
        if pair_str != str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']):

          # Viene salvato l'end time
          e_time = task_activity.iloc[i + 1]['time']

          ins_upd = True
          pair_str_prev = pair_str
        
        else:
          ins_upd = False
          pair_str_prev = pair_str
      
      else:

        # Viene salvato l'end time
        e_time = end_time

        ins_upd = True
        pair_str_prev = pair_str

    # Controlla se start ed end time sono stati trovati
    if ins_upd:

      # Controlla se il task-activity attuale non è l'ultimo
      if not last_node:

        # Aggiorna il numero di edge del grafo
        num_edge = num_edge + 1

        # Controlla se esiste un arco per il task-activity attuale 
        if pair_str in res_edge.index:

          # Aggiunge start ed end time all'arco
          res_edge.loc[pair_str, 'times'] += [((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]
        
        else:
          # Crea un nuovo arco
          res_edge.loc[pair_str] = [[((s_time, e_time), str(task_activity.iloc[i + 1]['task']) + '-' + str(task_activity.iloc[i + 1]['activity']))]]

      # Controlla se non esiste un nodo per il task-activity attuale
      if not pair_str in res_node.index:

        # Crea un array vuoto per le fueatures del nodo
        features_node = np.zeros(dim_features, dtype=np.int8)

        # Seleziona i sensori utilizzati per il task-activity attuale
        task_activity_sensors = sensors_events.query('time >= "' + s_time + '" and time <= "' + e_time + '"')

        # Popola l'array delle features
        for _, s in task_activity_sensors.iterrows():
          features_node[dict_sens[s['sensor']]] = 1

        # Salva nel nodo l'array delle features
        res_node.loc[pair_str] = [features_node]
  
  return res_edge, res_node, num_edge

In [110]:
sensors = pd.read_csv('drive/MyDrive/CASAS400/sensors.csv')
events = pd.read_csv('drive/MyDrive/CASAS400/events.csv')
tasks = pd.read_csv('drive/MyDrive/CASAS400/tasks.csv')
activities = pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

patients = pd.read_csv('drive/MyDrive/CASAS400/patients.csv')['patient_id']

dataset = []

for i in range(0, len(patients)):
  # Crea i dataframe con le features degli edges e dei nodi
  edge, node, num_edge = task_activity_sensors(tasks, events, sensors, activities, str(patients[i]))

  atr_node = np.zeros([len(node), len(node.iloc[0]['features'])])

  # Converte il dataframe dei nodi in una matrice
  for j in range(0, len(node)):
    atr_node[j] = node.iloc[j]['features']

  adj_matrix = np.empty([num_edge, 2])
  index_adj_matrix = 0

  # Crea la matrice di addiacenza dei nodi
  for j in range(0, len(edge)):
    for k in range(0, len(edge.iloc[j]['times'])):
      f_node = node.index.get_loc(edge.iloc[j].name)
      s_node = node.index.get_loc(edge.iloc[j]['times'][k][1])

      adj_matrix[index_adj_matrix] = np.array([f_node, s_node])
      index_adj_matrix = index_adj_matrix + 1
  
  # Salva la mtrice delle features e la matrice di addiacenza del grafo
  dataset.append(Data(x=torch.tensor(atr_node, dtype=torch.float),
                      edge_index=torch.tensor(adj_matrix, dtype=torch.long).t().contiguous()))

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/events.csv')

,patient,time,sensor,value
0,88,09:00:05.78693,m010,on
1,88,09:00:08.08177,m010,off
2,88,09:00:13.71834,m013,off
3,88,09:00:19.76291,m010,on
4,88,09:00:21.6229,m013,on
...,...,...,...,...
1209934,329,16:20:29.92485,m022,off
1209935,329,16:20:32.888,m021,off
1209936,329,16:20:32.91215,m025,off
1209937,329,16:20:33.9529,d001,close


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/activities.csv')

,patient,activity_type,start,end
0,88,1,09:21:17.2387,09:24:10.78761
1,88,2,09:25:15.5674,09:28:42.21513
2,88,3,09:30:21.23501,09:33:50.33897
3,88,4,09:34:53.77164,09:42:28.82678
4,88,5,09:43:36.54254,09:44:33.83539
...,...,...,...,...
5274,329,11,14:36:47.11877,14:40:02.51782
5275,329,12,14:44:22.30159,14:47:07.77214
5276,329,13,14:48:19.93743,14:52:49.75592
5277,329,14,14:53:32.62822,15:02:49.75547


In [ ]:
pd.read_csv('drive/MyDrive/CASAS400/task_types.csv')

,activity_id,task_id,description,action_type
0,1,1,Participant retrieves broom from supply closet,1
1,1,2,Participant retrieves duster from supply closet,1
2,1,3,Participant retrieves dust pan and brush from ...,1
3,1,4,Participant sweeps kitchen floor,5
4,1,5,Participant uses dust pan and brush,5
...,...,...,...,...
104,16,3,Participant brings deck of cards to the dining...,5
105,16,4,Participant sets up card game,5
106,16,5,Participant answers the telephone,11
107,16,6,Participant explains the rules of the game ove...,5
